In [ ]:
# ================================================================
# 1. 환경 설정 및 라이브러리 설치
# ================================================================

# 필요한 라이브러리 설치
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers datasets peft accelerate bitsandbytes
!pip install wandb trl
!pip install --upgrade huggingface_hub


In [1]:
!pip install trl

In [2]:
import torch
import json
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
import wandb
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore")

print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"GPU 개수: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"현재 GPU: {torch.cuda.get_device_name()}")

CUDA 사용 가능: True
GPU 개수: 1
현재 GPU: Tesla T4


In [12]:
# TRL이 설치되지 않은 경우를 대비한 대안
try:
    from trl import SFTTrainer
    USE_SFT = True
    print("SFTTrainer를 사용합니다.")
except ImportError:
    print("TRL이 설치되지 않았습니다. 기본 Trainer를 사용합니다.")
    USE_SFT = False

SFTTrainer를 사용합니다.


In [3]:
# Hugging Face 로그인 (필수)
from huggingface_hub import login
from google.colab import userdata

# 토큰 입력 (보안을 위해 숨겨서 입력)
login(token=userdata.get('HF_TOKEN'))
print("✅ Hugging Face 로그인 완료!")

✅ Hugging Face 로그인 완료!


In [4]:
# ================================================================
# 2. 모델 및 토크나이저 로드
# ================================================================

model_name = "microsoft/phi-2"
print(f"모델 로딩 중: {model_name}")

# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 4bit 양자화 설정 (메모리 효율성을 위해)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16
)

# 모델을 kbit 훈련용으로 준비
model = prepare_model_for_kbit_training(model)

print("모델 로딩 완료!")

모델 로딩 중: microsoft/phi-2


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

모델 로딩 완료!


In [5]:
# ================================================================
# 3. LoRA 설정
# ================================================================

# LoRA 구성
lora_config = LoraConfig(
    r=16,                   # rank
    lora_alpha=32,          # alpha scaling
    target_modules=[        # Phi-2의 attention 모듈들
        "q_proj",
        "k_proj",
        "v_proj",
        "dense",
        "fc1",
        "fc2"
    ],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# LoRA 어댑터 추가
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 23,592,960 || all params: 2,803,276,800 || trainable%: 0.8416


In [6]:
# ================================================================
# 4. 데이터셋 준비 (예제 - 실제 QA 데이터로 교체하세요)
# ================================================================
qa_data = json.load(open('qa_pairs.json', 'r', encoding='utf-8'))

def format_qa_pair(example):
    """QA 쌍을 훈련용 텍스트로 포맷팅"""
    question = example['question']
    answer = example['answer']
    context = example['context']
    # Context를 포함한 Phi-2에 적합한 프롬프트 템플릿
    formatted_text = f"Context: {context}\n\nQuestion: {question}\nAnswer: {answer}<|endoftext|>"

    return {"text": formatted_text}

# 데이터셋 생성
dataset = Dataset.from_list(qa_data)
dataset = dataset.map(format_qa_pair)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

데이터셋 크기: 1000
데이터 예시:
Context: Help us improve our services
We’re looking for people to try out our products. Find out how you
can participate.
Top questions about visiting Canada
How long can I stay in Canada as a visitor?
What is the difference between a single and a multiple
entry visa?
How do I help a family member or friend apply to visit
Canada?
See all questions about this topic
Date modified:
2025-05-07
25. 5. 12. 오전 3:24
Visitor Visa: Prepare for your arrival - Canada.ca
https://www.canada.ca/en/immigration-refugees-citizenship/services/visit-canada/prepare-arrival.html
4/4

---

visitor visa or eTA
work permit
study permit
You can submit all applications for your family with your work permit
application.
25. 5. 12. 오전 3:55
Who can apply - Canada.ca
https://www.canada.ca/en/immigration-refugees-citizenship/services/immigrate-canada/start-visa/work-permits/eligibility.html
5/6

---

Get a visa
Visas are issued by foreign government offices in Canada. Requirements,
fees and proc

In [7]:
print(f"데이터셋 크기: {len(dataset)}")
print("데이터 예시:")
print(dataset[999]["text"])

데이터셋 크기: 1000
데이터 예시:
Context: you’ll be allowed to export funds for all of your expenses.
How much money you need
All provinces/territories except Quebec
Number of family members
(including the applicant)
Amount of funds required per
year (not including tuition)
1
CAN$20,635
2
CAN$25,690
3
CAN$31,583
4
CAN$38,346
25. 5. 12. 오전 4:26
Study permit: Get the right documents - Proof of financial support - Canada.ca
https://www.canada.ca/en/immigration-refugees-citizenship/services/study-canada/study-permit/get-documents/financial-support.html
2/4

---

Help us improve our services
We’re looking for people to try out our products. Find out how you
can participate.
Top questions about visiting Canada
How long can I stay in Canada as a visitor?
What is the difference between a single and a multiple
entry visa?
How do I help a family member or friend apply to visit
Canada?
See all questions about this topic
Date modified:
2025-05-07
25. 5. 12. 오전 3:24
Visitor Visa: Prepare for your arrival - Ca

In [9]:
# ================================================================
# 5. 토크나이징 함수
# ================================================================

def tokenize_function(examples):
    """텍스트를 토큰화하는 함수"""
    # 텍스트가 리스트인지 문자열인지 확인
    texts = examples["text"] if isinstance(examples["text"], list) else [examples["text"]]

    tokenized = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors=None  # 배치 처리를 위해 None으로 설정
    )

    # labels를 input_ids와 동일하게 설정 (Causal Language Modeling)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# 데이터셋 토크나이징
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names
)

print("토크나이징 완료!")


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

토크나이징 완료!


In [15]:
# ================================================================
# 7. 훈련 설정
# ================================================================

# 데이터 콜레이터
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal LM이므로 False
)

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir="./phi2-lora-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=2,  # 메모리에 따라 조정
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    warmup_steps=100,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to=None,  # wandb 사용하려면 "wandb"로 변경
    load_best_model_at_end=False,
    dataloader_pin_memory=False,
    optim="paged_adamw_8bit",  # 메모리 효율적인 옵티마이저
    lr_scheduler_type="cosine",
    bf16=True,  # mixed precision
    ddp_find_unused_parameters=False,
)

# 트레이너 설정
if USE_SFT:
    # SFTTrainer 사용 (TRL이 설치된 경우) - 최신 API 사용
    try:
        trainer = SFTTrainer(
            model=model,
            train_dataset=dataset,  # 원본 데이터셋 사용
            args=training_args,
            tokenizer=tokenizer,
            packing=False,
        )
        print("✅ SFTTrainer 설정 완료")
    except Exception as e:
        print(f"SFTTrainer 오류: {e}")
        print("기본 Trainer로 전환합니다.")
        USE_SFT = False

if not USE_SFT:
    # 기본 Trainer 사용
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    print("✅ 기본 Trainer 설정 완료")

print("트레이너 설정 완료!")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


✅ 기본 Trainer 설정 완료
트레이너 설정 완료!


In [16]:
# ================================================================
# 7. 훈련 실행
# ================================================================

print("훈련 시작...")
trainer.train()

print("훈련 완료!")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


훈련 시작...


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cometlee39 (cometlee39-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,2.007900
20,1.958500
30,1.835600
40,1.476100
50,1.115200
60,0.789400
70,0.622500
80,0.461700
90,0.405500
100,0.297400


Step,Training Loss
10,2.007900
20,1.958500
30,1.835600
40,1.476100
50,1.115200
60,0.789400
70,0.622500
80,0.461700
90,0.405500
100,0.297400


훈련 완료!


In [17]:
# ================================================================
# 8. 모델 저장
# ================================================================

# LoRA 어댑터만 저장 (전체 모델보다 훨씬 작음)
model.save_pretrained("./phi2-lora-adapter")
tokenizer.save_pretrained("./phi2-lora-adapter")

print("모델 저장 완료!")

모델 저장 완료!


In [25]:
# ================================================================
# 10. 추론 테스트
# ================================================================

def generate_response(question, context=None):
    """질문에 대한 답변 생성 - 개선된 버전"""

    # 훈련 데이터와 동일한 형식으로 프롬프트 구성
    if context:
        prompt = f"Context: {context}\n\nQuestion: {question}\nAnswer:"
    else:
        # Context 없이 추론할 때는 간단한 지시문 추가
        prompt = f"Please answer the following question accurately and concisely.\n\nQuestion: {question}\nAnswer:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=120,  # max_length 대신 max_new_tokens 사용
            num_return_sequences=1,
            temperature=0.1,  # 더 낮은 온도로 일관성 향상
            top_p=0.9,  # nucleus sampling 추가
            do_sample=True,
            repetition_penalty=1.1,  # 반복 방지
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=3,  # 3-gram 반복 방지
            # 문장 완성을 위한 추가 설정
            length_penalty=1.2,  # 긴 답변 선호
            early_stopping=False  # 조기 종료 방지
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 프롬프트 부분 제거하고 답변만 반환
    answer = response.split("Answer:")[-1].strip()

    # 불필요한 부분 정리
    if "<|endoftext|>" in answer:
        answer = answer.split("<|endoftext|>")[0].strip()

    return answer

# 테스트 질문들 (context 포함/미포함 버전)
test_cases = [
   {
    "question": "What types of visas are available for Canada?",
    "context": "Help us improve our services\nWe’re looking for people to try out our products. Find out how you\ncan participate.\nTop questions about visiting Canada\nHow long can I stay in Canada as a visitor?\nWhat is the difference between a single and a multiple\nentry visa?\nHow do I help a family member or friend apply to visit\nCanada?\nSee all questions about this topic\nDate modified:\n2025-05-07\n25. 5. 12. 오전 3:24\nVisitor Visa: Prepare for your arrival - Canada.ca\nhttps://www.canada.ca/en/immigration-refugees-citizenship/services/visit-canada/prepare-arrival.html\n4/4\n\n---\n\nvisitor visa or eTA\nwork permit\nstudy permit\nYou can submit all applications for your family with your work permit\napplication.\n25. 5. 12. 오전 3:55\nWho can apply - Canada.ca\nhttps://www.canada.ca/en/immigration-refugees-citizenship/services/immigrate-canada/start-visa/work-permits/eligibility.html\n5/6\n\n---\n\nGet a visa\nVisas are issued by foreign government offices in Canada. Requirements,\nfees and processing times vary, depending on"
  },
  {
    "question": "How do I apply for a visa to Canada?",
    "context": "You may need to complete extra steps when you fill out your\napplication.\nHow to apply for your new visitor visa\nYou need to apply online in your IRCC secure account. If you don’t\nalready have an account, follow these steps to register.\nYou’ll need to select “Apply to Come to Canada” from your account main\npage to get started.\nGet the right application form\nTo get the right application form, provide these answers in the online\nquestionnaire:\nFor the first question “What would you like to do in Canada?”, select\n“Study” if you have a valid study permit or\n“Work” if you have a valid work permit\nWhen asked “What is your current country/territory of residence?”,\nselect “Canada”.\nAnswer the questions on the next pages about your work or studies.\nYou may be given the option to extend your current study or work\npermit or to apply for a “Temporary Resident Visa”. Make sure you\nselect “Temporary Resident Visa”.\nThe application form listed in your document checklist will be called\n\n---\n\nGet a visa"
  },
  {
    "question": "What documents are required for a Canada visa?",
    "context": "Prepare your documents\nWhen you arrive in Canada, you must have\nyour passport\nyour visitor visa (if you need one)\nany travel document(s) you’re carrying\nYou may also need to show proof that your work permit application was\napproved. One way to do this is to show the border services officer your\nport of entry letter of introduction. You’ll get this letter if you\napplied online or\ngave us an email address in the work permit application form for\ncommunicating with you \nYou can print this letter or bring an electronic version with you.\nYou should also bring supporting documents, such as\nproof that you meet the requirements of the job, such as proof of\nwork experience and education\na copy of your employer’s positive labour market impact assessment\n(LMIA), if required\nYou’ll also need a copy of your attestation of issuance of your\nQuebec Acceptance Certificate (CAQ), if your employer needed\nan LMIA and you’ll work in Quebec.\nthe offer of employment number your employer received when they\n\n--"
  },
  {
    "question": "How much does a Canada visa cost?",
    "context": "Canada.ca\n \nTravel\n \nTravel abroad\n \nTravel documents\n\n\n\nVisas, biometrics and electronic travel\nauthorizations\nEntry requirements vary from country to country. Before you travel,\nunderstand what you need to enter the destination including any visas,\nelectronic travel authorizations (ETAs) and/or your biometrics.\nOn this page\nVisas\nBiometrics\nElectronic travel authorizations\nVisas\nA visa is an official document, usually stamped or glued inside a\npassport, that allows a foreign national to enter a country for a specific\npurpose and for a set amount of time.\nFind out if you need a visa\nTo find out if you need any visas for your trip, start by consulting our\nTravel advice and advisories page well before you plan to leave. Select\nyour destination from the drop-down menu and consult the information\non visas under “Entry and exit requirements”.\nYou should then contact your destination’s embassy or consulate in\nCanada to confirm whether you need a visa to enter the country or stay\n\n---\n\nHo"
  },
  {
    "question": "What is the processing time for a Canada visa?",
    "context": "You may need to give biometrics with your application. This\nprocessing time doesn’t include the time you need to give\nbiometrics.\n\n25. 5. 12. 오전 3:24\nTransit visa: After you apply - Canada.ca\nhttps://www.canada.ca/en/immigration-refugees-citizenship/services/visit-canada/transit/transit-visa/after-apply-next-steps.html\n2/3\n\n---\n\nCanada class and you’ve submitted an application for an open work\npermit, your work permit will normally be processed within four months.\n25. 5. 12. 오전 4:14\nSponsor your spouse, common-law partner, conjugal partner or dependent child – Complete Guide (IMM 5289) - Canada.ca\nhttps://www.canada.ca/en/immigration-refugees-citizenship/services/application/application-forms-guides/guide-5289-sponsor-your-spouse-co…\n71/89\n\n---\n\nGet a visa\nVisas are issued by foreign government offices in Canada. Requirements,\nfees and processing times vary, depending on the country and type of\nvisa you need. The most common categories are business, work, student\nand tourist visas. Ge"
  }
]

print("\n=== 개선된 추론 테스트 ===")
for i, test_case in enumerate(test_cases, 1):
    question = test_case["question"]
    context = test_case["context"]

    # 다양한 파라미터 조합 테스트
    print(f"\n[테스트 {i}]")
    if context:
        print(f"Context: {context[:100]}...")
    print(f"Q: {question}")

    answer = generate_response(
            question,
            context
        )
    print(f"A: {answer}")
    print("-" * 60)


=== 개선된 추론 테스트 ===

[테스트 1]
Context: Help us improve our services
We’re looking for people to try out our products. Find out how you
can ...
Q: What types of visas are available for Canada?
A(보수적): For Canada, there are different types of visitors' visas available such as business visas, work permits, student visas, and tourist visas. The specific requirements, fees, and processing time for each type of visa may vary based on the country and the purpose of your visit. It's important to check the travel advice for your destination and contact the foreign government office in Canada for accurate information on visa application procedures. Remember to follow the guidelines provided by the Canadian authorities to ensure a smooth and successful visa process.
A(균형): The most common types of Visas available for Canadians include tourist visas, business visas, student visas, and work permits. Each type of visa has specific requirements, fees, and processing timelines that depend on the countr

In [1]:
# ================================================================
# 10. 선택사항: 허깅페이스 허브에 업로드
# ================================================================

# 모델 업로드
model.push_to_hub("cometlee39/phi2-lora-finetuned")
tokenizer.push_to_hub("cometlee39/phi2-lora-finetuned")

print("\n모든 과정이 완료되었습니다!")
print("LoRA 어댑터가 './phi2-lora-adapter' 폴더에 저장되었습니다.")


NameError: name 'model' is not defined